# <center><span style="color:#336699">Introdução à Programação com Dados Geoespaciais em Ambientes de Computação Interativa</span></center>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 150%;">
    Aula 02: Manipulação de Dados Vetoriais em Python</br>
    <span style="font-size: 0.75em;">Parte II - Sistemas de Referência de Coordenadas</span>
</div>

<br/>

<div style="text-align: center;font-size: 90%;">
    Gilberto Ribeiro de Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Karine Reis Ferreira<sup><a href="https://orcid.org/0000-0003-2656-5504"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Marcos Adami<sup><a href="https://orcid.org/0000-0003-4247-4477"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Thales Sehn Körting<sup><a href="https://orcid.org/0000-0002-0876-0501"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Divisão de Observação da Terra e Geoinformática, Instituto Nacional de Pesquisas Espaciais (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Última Atualização: 30 de Janeiro de 2025
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
    <b>Resumo.</b> Este Jupyter Notebook apresenta uma visão geral de como trabalhar com sistemas de referência espacial em Python.
</div>

# <span style="color:#336699">Sistemas de Referência de Coordenadas</span>
<hr style="border:1px solid #0077b9;">

Para representar a Terra em uma superfície bidimensional de um mapa, é necessário que as coordenadas dos objetos geográficos estejam associadas a um **sistema de referência de coordenadas** (**coordinate reference system** ou **CRS**). Basicamente, no dia a dia, lidamos com dois tipos de sistemas:

- **Sistemas de Coordendas Geográficas (Geographic Coordinate System ou GCS):** que utiliza um modelo esférico onde as localizações são baseadas em valores de longitude e latitude. Existem diversos GCS, sendo o `WGS84` (*World Geodetic System 1984*) um dos mais conhecidos e usados. No Brasil, diversas instituições adotam o sistema `SIRGAS 2000` na distribuição de seus dados.

- **Sistema de Coordenadas Projetadas (Projected Coordinate System ou PCS):** esse tipo de sistema também conhecido como **projeção cartográfica** (**map projection**) são baseados em modelos voltados para desenhar mapas numa superfície plana. São exemplos deste tipo de sistema o `WGS 84 / Pseudo-Mercator`, usado por aplicativos Google Maps e OpenStreetMap, e o CRS Azimutal Equidistante.


Os CRS também são conhecidos nos sistemas de informação geográficas por códigos definidos por autoridades como o EPSG (European Petroleum Survey Group) e a ESRI:

- `EPSG:4326`: WGS 84 (GCS).

- `EPSG:4674`: SIRGAS 2000 (GCS).

- `EPSG:3857`: WGS 84 / Pseudo-Mercator (PCS).

- `ESRI:54032`: World Azimuthal Equidistant (PCS).

- `ESRI:54017`: World Behrmann, Projeção Cilíndrica Equalarea (PCS).


Existem várias formas de descrição dos sistemas de referência de coordenadas, sendo as mais conhecidas os formatos `ESRI WKT`, o formato adotado em arquivos `.prj` do conjunto ESRI Shapefile, o `PROJJSON` e o formato da biblioteca `PROJ`. Por exemplo, o CRS `WGS 84 / Pseudo-Mercator` poderia ser representado nos formatos abaixo com as seguintes strings:

- ESRI WKT (*Well Know Text*):
  ```text
  PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",
         GEOGCS["GCS_WGS_1984",
                DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],
                PRIMEM["Greenwich",0.0],
                UNIT["Degree",0.0174532925199433]
         ],
         PROJECTION["Mercator_Auxiliary_Sphere"],
         PARAMETER["False_Easting",0.0],
         PARAMETER["False_Northing",0.0],
         PARAMETER["Central_Meridian",0.0],
         PARAMETER["Standard_Parallel_1",0.0],
         PARAMETER["Auxiliary_Sphere_Type",0.0],
         UNIT["Meter",1.0]
  ]
  ```

- PROJ:
  ```text
  +proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs +type=crs
  ```

  
O site https://epsg.io pode ser utilizado para consultas online a uma ampla base de dados sobre sistemas de referência de coordenadas. Na guia `Map` é possível alterar o CRS usado para indicar a localização sobre o mapa de referência. 


Outro site contendo uma base de dados sobre sistemas de referência de coordenadas é o https://spatialreference.org. Nele, além da descrição do CRS, é possível obter a codificação de um dado sistema nos formatos `ESRI WKT`, do formato adotado em arquivos `.prj` do conjunto ESRI Shapefile, `PROJJSON` e o formato da biblioteca `PROJ`.


Esta seção servirá como um primeiro contato com o tópico de **sistemas de referência de coordenadas** e será detalhado mais adiante no curso.

<img src="./img/logo/pyproj.png" align="right" width="192" />

# <span style="color:#336699">Usando a biblioteca pyproj</span>
<hr style="border:1px solid #0077b9;">

No mundo de software livre, existe uma biblioteca C/C++ denominada **[PROJ](https://github.com/OSGeo/PROJ)** que fornece funcionalidades para realizar conversões entre projeções cartográficas. Essa biblioteca, cuja base de código vem dos anos 80 (ver [Wikipedia](https://en-wikipedia-org.translate.goog/wiki/PROJ)), é utilizada em diversos sistemas, como no QGIS e PostGIS. Ela possui uma interface em Python denominada **[pyproj](https://pyproj4.github.io/pyproj/stable/)**, que pode ser instalada com o seguinte comando:

```bash
conda install --channel conda-forge pyproj
```


Nesta seção iremos utilizar a biblioteca `pyproj` para explorar transformações entre diferentes sistemas de referência espacial na linguagem Python.

Para acessar as funconalidades dessa biblioteca devemos importá-la em nosso código:

In [ ]:
import pyproj

Vamos verificar a versão instalada:

In [ ]:
pyproj.__version__

A classe `CRS` permite criar objetos para lidar com sistemas conhecidos, como o caso dos CRSs `EPSG:4326` ou `EPSG:4674`:

In [ ]:
crs_epsg_4326 = pyproj.CRS.from_epsg(4326)
crs_epsg_4326

In [ ]:
crs_epsg_4674 = pyproj.CRS.from_epsg(4674)
crs_epsg_4674

Repare na saída das duas células de código anteriores que esses CRSs são denominados *Geographic 2D CRS*.

Na célula seguinte, iremos criar um CRS baseado no sistema identificado pelo código `ESRI:54032`. Para tal, usaremos outro método da classe CRS: 

In [ ]:
crs_esri_54032 = pyproj.CRS.from_string("ESRI:54032")
crs_esri_54032

Repare que a saída agora apresenta esse CRS como *Projected CRS*.

Também podemos criar um objeto CRS através de uma string no formato da biblioteca PROJ. A célula seguinte cria um CRS baseado nos parâmetros utilizados pelos produtos do [Brazil Data Cube](https://data.inpe.br/bdc). Trata-se de um sistema personalizado, baseado em uma projeção *Albers Equal Area*:

In [ ]:
crs_bdc = pyproj.CRS.from_proj4("+proj=aea +lat_1=-2 +lat_2=-22 +lat_0=-12 +lon_0=-54 +x_0=5000000 +y_0=10000000 +ellps=GRS80 +units=m +no_defs ")
crs_bdc

A transaformação entre sistemas de coordenadas pode ser facilmente realizada com a classe `Transformer`. A célula abaixo irá transformar as coordenadas de longitude -45.861247 e latitude -23.207336 no CRS `EPSG:4326` para o `ESRI:54032`:

In [ ]:
transformer = pyproj.Transformer.from_crs(crs_epsg_4326, crs_esri_54032, always_xy=True)
transformer

In [ ]:
transformer.transform(-45.861247, -23.207336)

A biblioteca `pyproj` também fornece funcionalidades básicas para cálculos geodésicos de distância entre dois pontos ou a áreas de um polígono sobre a superfície terrestre.

In [ ]:
lon1, lat1 = -45.861247, -23.207336
lon2, lat2 = -43.230085, -22.912172

In [ ]:
geod = crs_epsg_4326.get_geod()
geod

In [ ]:
# Calculate distance
# inv returns (forward_azimuth, back_azimuth, distance_in_meters)
azi12, azi21, distance = geod.inv(lon1, lat1, lon2, lat2)
distance

In [ ]:
azi12

In [ ]:
azi21

Para saber mais sobre as classes `CRS` e `Transformer`, utilize as células abaixo:

In [ ]:
#pyproj.CRS?

In [ ]:
#pyproj.Transformer?

# <span style="color:#336699">Referências Bibliográficas</span>
<hr style="border:1px solid #0077b9;">

- [Documentação da biblioteca pyproj](https://pyproj4.github.io/pyproj/stable/). Disponível online.

- [Latitude/Longitude Distance Calculator](https://www.nhc.noaa.gov/gccalc.shtml)